## Working with Geospatial Data

In [1]:
# Import packages
import requests
import folium
import urllib
from pandas.io.json import json_normalize
import pandas as pd
from geopandas import GeoDataFrame
from shapely.geometry import Point


In [14]:
# Get URL
water_base_url = "https://data.colorado.gov/resource/j5pc-4t32.json?"
water_full_url = water_base_url + "station_status=Active" + "&county=BOULDER"


In [15]:
water_full_url

'https://data.colorado.gov/resource/j5pc-4t32.json?station_status=Active&county=BOULDER'

In [16]:
data = requests.get(water_full_url)

In [10]:
type(data.json())


list

In [17]:
data.json()[:2]

[{'station_name': 'PECK-PELLA AUGMENTATION RETURN',
  'div': '1',
  'location': {'latitude': '40.160705',
   'needs_recoding': False,
   'longitude': '-105.168471'},
  'dwr_abbrev': 'PECRTNCO',
  'data_source': 'Co. Division of Water Resources',
  'amount': '0.00',
  'station_type': 'Diversion',
  'wd': '5',
  'http_linkage': {'url': 'https://dwr.state.co.us/Tools/Stations/PECRTNCO'},
  'date_time': '2022-02-07T10:00:00.000',
  'county': 'BOULDER',
  'variable': 'DISCHRG',
  'station_status': 'Active'},
 {'station_name': 'HIGHLAND DITCH AT LYONS, CO',
  'div': '1',
  'location': {'latitude': '40.215043',
   'needs_recoding': False,
   'longitude': '-105.256017'},
  'dwr_abbrev': 'HIGHLDCO',
  'data_source': 'Co. Division of Water Resources',
  'amount': '0.00',
  'station_type': 'Diversion',
  'wd': '5',
  'http_linkage': {'url': 'https://dwr.state.co.us/Tools/Stations/HIGHLDCO'},
  'date_time': '2022-02-07T09:30:00.000',
  'county': 'BOULDER',
  'variable': 'DISCHRG',
  'stage': '-0.0

## Convert JSON to Pandas DataFrame

In [18]:
from pandas.io.json import json_normalize

In [20]:
result = json_normalize(data.json())

C:\Users\fnasr\AppData\Local\Temp\ipykernel_17488\3462806043.py:1: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead.
  result = json_normalize(data.json())


In [21]:
result.head()

,station_name,div,dwr_abbrev,data_source,amount,station_type,wd,date_time,county,variable,station_status,location.latitude,location.needs_recoding,location.longitude,http_linkage.url,stage,usgs_station_id,flag
0,PECK-PELLA AUGMENTATION RETURN,1,PECRTNCO,Co. Division of Water Resources,0.00,Diversion,5,2022-02-07T10:00:00.000,BOULDER,DISCHRG,Active,40.160705,False,-105.168471,https://dwr.state.co.us/Tools/Stations/PECRTNCO,NaN,NaN,NaN
1,"HIGHLAND DITCH AT LYONS, CO",1,HIGHLDCO,Co. Division of Water Resources,0.00,Diversion,5,2022-02-07T09:30:00.000,BOULDER,DISCHRG,Active,40.215043,False,-105.256017,https://dwr.state.co.us/Tools/Stations/HIGHLDCO,-0.07,NaN,NaN
2,"BOULDER CREEK AT NORTH 75TH ST. NEAR BOULDER, CO",1,BOCNORCO,U.S. Geological Survey,5.96,Stream,6,2022-01-10T15:15:00.000,BOULDER,DISCHRG,Active,40.051655,False,-105.178883,https://dwr.state.co.us/Tools/Stations/BOCNORCO,NaN,06730200,NaN
3,PALMERTON DITCH,1,PALDITCO,"Cooperative Program of CDWR, NCWCD & SVLHWCD",0.00,Diversion,5,2022-02-07T10:00:00.000,BOULDER,DISCHRG,Active,40.212511,False,-105.251836,https://dwr.state.co.us/Tools/Stations/PALDITCO,NaN,NaN,NaN
4,LITTLE THOMPSON #2 DITCH,1,LITTH2CO,Northern Water,0.00,Diversion,4,2022-02-07T10:00:00.000,BOULDER,DISCHRG,Active,40.255787,False,-105.209511,https://dwr.state.co.us/Tools/Stations/LITTH2CO,0.00,ES1900,NaN


In [22]:
type(result)

pandas.core.frame.DataFrame

In [23]:
result.columns

Index(['station_name', 'div', 'dwr_abbrev', 'data_source', 'amount',
       'station_type', 'wd', 'date_time', 'county', 'variable',
       'station_status', 'location.latitude', 'location.needs_recoding',
       'location.longitude', 'http_linkage.url', 'stage', 'usgs_station_id',
       'flag'],
      dtype='object')

In [24]:
result['location.latitude'][0]

'40.160705'

In [25]:
result['location.latitude'] = result['location.latitude'].astype(float)

In [26]:
result['location.latitude'][0]

40.160705

In [28]:
result['location.longitude'] = result['location.longitude'].astype(float)

In [29]:
result.shape


(74, 18)

In [30]:
result['location.latitude'].isna().any()

False

In [31]:
result['location.longitude'].isna().any()

False

## Data Visualization


In [34]:
geometry = [Point(xy) for xy in zip(result['location.longitude'], result['location.latitude'])]
crs = {'init': 'epsg:4326'}
gdf = GeoDataFrame(result, crs=crs, geometry=geometry)

C:\Users\fnasr\Miniconda3\envs\earth-analytics-python\lib\site-packages\pyproj\crs\crs.py:131: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)


In [35]:
gdf.head()

,station_name,div,dwr_abbrev,data_source,amount,station_type,wd,date_time,county,variable,station_status,location.latitude,location.needs_recoding,location.longitude,http_linkage.url,stage,usgs_station_id,flag,geometry
0,PECK-PELLA AUGMENTATION RETURN,1,PECRTNCO,Co. Division of Water Resources,0.00,Diversion,5,2022-02-07T10:00:00.000,BOULDER,DISCHRG,Active,40.160705,False,-105.168471,https://dwr.state.co.us/Tools/Stations/PECRTNCO,NaN,NaN,NaN,POINT (-105.16847 40.16071)
1,"HIGHLAND DITCH AT LYONS, CO",1,HIGHLDCO,Co. Division of Water Resources,0.00,Diversion,5,2022-02-07T09:30:00.000,BOULDER,DISCHRG,Active,40.215043,False,-105.256017,https://dwr.state.co.us/Tools/Stations/HIGHLDCO,-0.07,NaN,NaN,POINT (-105.25602 40.21504)
2,"BOULDER CREEK AT NORTH 75TH ST. NEAR BOULDER, CO",1,BOCNORCO,U.S. Geological Survey,5.96,Stream,6,2022-01-10T15:15:00.000,BOULDER,DISCHRG,Active,40.051655,False,-105.178883,https://dwr.state.co.us/Tools/Stations/BOCNORCO,NaN,06730200,NaN,POINT (-105.17888 40.05165)
3,PALMERTON DITCH,1,PALDITCO,"Cooperative Program of CDWR, NCWCD & SVLHWCD",0.00,Diversion,5,2022-02-07T10:00:00.000,BOULDER,DISCHRG,Active,40.212511,False,-105.251836,https://dwr.state.co.us/Tools/Stations/PALDITCO,NaN,NaN,NaN,POINT (-105.25184 40.21251)
4,LITTLE THOMPSON #2 DITCH,1,LITTH2CO,Northern Water,0.00,Diversion,4,2022-02-07T10:00:00.000,BOULDER,DISCHRG,Active,40.255787,False,-105.209511,https://dwr.state.co.us/Tools/Stations/LITTH2CO,0.00,ES1900,NaN,POINT (-105.20951 40.25579)


In [37]:
m = folium.Map([40.01, -105.27], zoom_start=10, tiles='cartodbpositron')
folium.GeoJson(gdf).add_to(m)
m

In [39]:
# Get the latitude and longitude from result as a list
locations = result[['location.latitude', 'location.longitude']]
coords = locations.values.tolist()

In [43]:
from folium.plugins import MarkerCluster

m = folium.Map([40.01, -105.27], zoom_start= 10, tiles='cartodbpositron')

marker_cluster = MarkerCluster().add_to(m)

for point in range(0, len(coords)):
    folium.Marker(location = coords[point], popup= 'Name: ' + result['station_name'][point] + ' ' + 'Precip: ' + str(result['amount'][point])).add_to(marker_cluster)

m